**Twitter Elections Jake**<br>
*Compsci 209 Final Project*

Follow the instructions here for pulling tweets:<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/premium-search#DataEndpoint

Tutorial for pulling geography of tweets:<br>
https://stackabuse.com/accessing-the-twitter-api-with-python/

Query Parameter:<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

----

In [1]:
from twython import Twython
import pandas as pd
import sys

In [2]:
import config_twitter

app_key = config_twitter.api_key
app_secret_key = config_twitter.api_secret_key
access_token = config_twitter.access_token
access_token_secret = config_twitter.access_token_secret

user_agent = "Jake_Schneider"

In [3]:
twython = Twython(app_key, app_secret_key,
                  access_token, access_token_secret)

In [4]:
twython.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1575675984}}

In [5]:
twython.verify_credentials()

{'id': 1293308917,
 'id_str': '1293308917',
 'name': 'Jake Schneider',
 'screen_name': '_jwschneider',
 'location': 'Cambridge, MA',
 'description': 'Economist. Data Scientist. Entrepreneur. \nFounder @SchneiderEconomics. Founder @PolicyViews. MPA/ID Candidate @Harvard @Kennedy_School.',
 'url': 'https://t.co/KiRNKCPigk',
 'entities': {'url': {'urls': [{'url': 'https://t.co/KiRNKCPigk',
     'expanded_url': 'http://www.policyviews.com',
     'display_url': 'policyviews.com',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 59,
 'friends_count': 171,
 'listed_count': 0,
 'created_at': 'Sun Mar 24 02:33:01 +0000 2013',
 'favourites_count': 48,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'verified': False,
 'statuses_count': 59,
 'lang': None,
 'status': {'created_at': 'Sun Nov 24 15:52:33 +0000 2019',
  'id': 1198630493694169089,
  'id_str': '1198630493694169089',
  'text': "@BarackObama Recently #Harvard's student 

----

**Search for Conservative Tweets**

In [6]:
# Create query
query = {'q': 'conservative',
        #'geocode': '37.0902, -95.7129, 1500mi',
        'result_type': 'popular',
        'count': 1000,
        'lang': 'en',
#        'until': "2018-11-06" 
        }

print(query)

{'q': 'conservative', 'result_type': 'popular', 'count': 1000, 'lang': 'en'}


In [7]:
twython.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1575675984}}

In [8]:
# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'user_loc': []}
for status in twython.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['user_loc'].append(status['user']['location'])


    
print(len(dict_["user"]))

15


In [9]:
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(100)

,user,date,text,favorite_count,user_loc
3,PickardJE,Fri Dec 06 11:06:27 +0000 2019,Tommy Robinson is endorsing Boris Johnson's Co...,19632,London
4,KTHopkins,Thu Dec 05 10:01:36 +0000 2019,My name is Katie Hopkins \n\nI am a straight. ...,19081,
2,DavidLammy,Fri Dec 06 13:59:47 +0000 2019,Huge blow for @Conservatives. Former Conservat...,16731,Tottenham
8,TalbertSwan,Thu Dec 05 12:58:24 +0000 2019,Melania hasn’t defended Black children murdere...,14837,Global
6,jeremycorbyn,Fri Dec 06 14:59:00 +0000 2019,Margaret Thatcher's Conservative government wa...,12689,UK
0,OwenJones84,Fri Dec 06 00:16:33 +0000 2019,Oh. Wow. Just wow. The Hastings &amp; Rye Cons...,12310,London
1,JackPosobiec,Thu Dec 05 12:54:15 +0000 2019,Adam Schiff is spying on the President’s laywe...,12132,"Washington, DC"
12,BorisJohnson,Thu Dec 05 20:47:00 +0000 2019,"Under my leadership, a majority Conservative g...",9226,United Kingdom
7,OwenJones84,Fri Dec 06 15:20:23 +0000 2019,A senior Conservative source tells me they tel...,7409,London
11,Peston,Fri Dec 06 12:49:08 +0000 2019,A Tory former prime minister Sir John Major do...,5350,


In [10]:
# Geographic Heat Map

from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim(user_agent = user_agent)

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(df.user_loc):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass
    
# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Save the map to html file
gmap.draw("/Users/jschneids13/Desktop/Harvard Kennedy School MPA-ID/Semesters/Semester 3, Fall 2019/COMPSCI 109 Data Science/Final Project/CS109a-Group-81-Final-Project/API/heat_map.html")

----

**Premium Query: Example "Conservative"** <br>
<br>
Resource from Luca Hammer: https://lucahammer.com/2019/11/05/collecting-old-tweets-with-the-twitter-premium-api-and-python/ <br>
<br>
Search Query Parameters from Fresh Van Roote: https://freshvanroot.com/blog/2019/twitter-search-guide-by-luca/

In [11]:
!pip install searchtweets

In [12]:
# Load OAUTH Bearer Token Credentials

from searchtweets import load_credentials

premium_search_args = load_credentials("twitter_keys_full_archive.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)
#print(premium_search_args)

/opt/anaconda3/lib/python3.7/site-packages/searchtweets/credentials.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  search_creds = yaml.load(f)[yaml_key]
Grabbing bearer token from OAUTH


In [68]:
# Create General Payload Rule

from searchtweets import gen_rule_payload

rule = gen_rule_payload("Conservative", 
                        results_per_call=100,
                        from_date="2018-08-06",
                        to_date="2018-11-06"
                       )

In [69]:
# Create Result Stream

from searchtweets import ResultStream

rs = ResultStream(rule_payload=rule,
                  max_results=100,
                  **premium_search_args)
print(rs)

ResultStream: 
	{
    "username": null,
    "endpoint": "https://api.twitter.com/1.1/tweets/search/fullarchive/elections.json",
    "rule_payload": {
        "query": "Conservative",
        "maxResults": 100,
        "toDate": "201811060000",
        "fromDate": "201808060000"
    },
    "tweetify": true,
    "max_results": 100
}


In [70]:
# Collect the Tweets

import json

with open('twitter_premium_api_demo_conservative.jsonl', 'a', encoding='utf-8') as f:
    n = 0
    for tweet in rs.stream():
        n += 1
        if n % 10 == 0:
            print('{0}: {1}'.format(str(n), tweet['created_at']))
        json.dump(tweet, f)
        f.write('\n')
print('done')

10: Mon Nov 05 23:59:53 +0000 2018
20: Mon Nov 05 23:59:44 +0000 2018
30: Mon Nov 05 23:59:38 +0000 2018
40: Mon Nov 05 23:59:28 +0000 2018
50: Mon Nov 05 23:59:22 +0000 2018
60: Mon Nov 05 23:59:07 +0000 2018
70: Mon Nov 05 23:59:03 +0000 2018
80: Mon Nov 05 23:58:47 +0000 2018
90: Mon Nov 05 23:58:37 +0000 2018
100: Mon Nov 05 23:58:33 +0000 2018
done


In [71]:
# Create Json Printer

import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [76]:
# Load Json File and Convert to PD Dataframe

tweets_conservative_df = pd.read_json('twitter_premium_api_demo_conservative.jsonl', lines=True)
tweets_conservative_df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,matching_rules,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet,possibly_sensitive,extended_entities
0,2019-12-06 21:04:29+00:00,1203057649039937537,1203057649039937536,RT @Conservatives: TRIPLE TAX LOCK: We will no...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-06 21:04:29+00:00,1203057648486232064,1203057648486232064,RT @RobBurl: There’s a certain irony to this. ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],1.202881e+18,1.202881e+18,{'created_at': 'Fri Dec 06 09:20:40 +0000 2019...,"{'url': 'https://t.co/UiQ2iVh299', 'expanded':...",NaN,NaN,NaN,NaN
2,2019-12-06 21:04:29+00:00,1203057647701901313,1203057647701901312,RT @SHKMEP: I am a Conservative proud Brit\nI ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],1.202955e+18,1.202955e+18,{'created_at': 'Fri Dec 06 14:18:15 +0000 2019...,"{'url': 'https://t.co/H08moq1Of1', 'expanded':...",NaN,NaN,NaN,NaN
3,2019-12-06 21:04:29+00:00,1203057645965340672,1203057645965340672,RT @imajsaclaimant: Disabled people put on a s...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-06 21:04:28+00:00,1203057644925272071,1203057644925272064,RT @BorisJohnson: If you work hard to provide ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# View Df Collumns
tweets_conservative_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'filter_level', 'lang', 'matching_rules',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'display_text_range', 'extended_tweet',
       'possibly_sensitive', 'extended_entities'],
      dtype='object')

In [78]:
# View Location Data through "GEO"
tweets_conservative_df[["geo", "coordinates", "place", "user"]].head()

,geo,coordinates,place,user
0,NaN,NaN,None,"{'id': 176217968, 'id_str': '176217968', 'name..."
1,NaN,NaN,None,"{'id': 2808839789, 'id_str': '2808839789', 'na..."
2,NaN,NaN,None,"{'id': 2175096381, 'id_str': '2175096381', 'na..."
3,NaN,NaN,None,"{'id': 147322918, 'id_str': '147322918', 'name..."
4,NaN,NaN,None,"{'id': 422238927, 'id_str': '422238927', 'name..."


In [88]:
# View User Information through "User"
for loc in range(0,100):
    print(tweets_conservative_df["user"][loc]["location"])

None
Manchester, England
None
Wolverhampton
None
Dartford, Kent
Neath, Wales
None
None
Liverpool, England
None
None
None
None
Portland, OR
New Jersey, USA
London
Cumbria, England
Nebraska, USA
Toronto, ON Canada 🍁
None
North West, England
London
None
Canada     I ❤️ 🍁 Oil & Gas
Manchester, England
Godalming
England
The place you'll never know
None
Boston, MA 🇺🇸
The Moon
London, England
None
London/Manchester
None
England, United Kingdom
United Kingdom
United Kingdom
Scotland, United Kingdom
North Essex
Liverpool
London, England
Hackney
Espanja
England, United Kingdom
None
England, United Kingdom
None
UK
None
Cotswolds & SW France
United Kingdom
Germany
United Kingdom
Keighley, England
None
United Kingdom
North Tyneside
Cheshire
None
Liam💖
None
Buttstädt, Deutschland
Herefordshire, UK
North West, England
England, United Kingdom
scotland | she/her
Liverpool, England
South East, England
Cambridge
Mordor / Bristol
Los Angeles, CA, USA
Bristol
None
Belfast
None
Finchley
Manchester, England


---

**Streamer**

In [ ]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            print(tweet_data)
#            self.save_to_csv(tweet_data)
            return tweet_data

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
#    def save_to_csv(self, tweet):
#        with open(r'saved_tweets_conservative.csv', 'a') as file:
#            writer = csv.writer(file)
#            writer.writerow(list(tweet.values()))

In [ ]:
## Instantiate from our streaming class
#stream = MyStreamer(app_key, app_secret_key,
#                  access_token, access_token_secret)
## Start the stream
#tweets = stream.statuses.filter(track='conservative')